In [19]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
from glob import glob

# Set the input folder and output folder
input_folder = r"C:\New folder\Desktop\segregated_top_left_defect\top_left_defect"
output_folder = r"C:\New folder\Desktop\segregated_top_left_defect\processed"

# Create output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Get all BMP images in the folder
image_paths = glob(os.path.join(input_folder, "*.PNG"))

# Processing loop
for image_path in image_paths:
    # Read image
    image = cv2.imread(image_path)
    if image is None:
        continue  # Skip if image couldn't be read

    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Estimate background using morphological operations
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (15, 15))
    background = cv2.morphologyEx(gray, cv2.MORPH_CLOSE, kernel)

    # Subtract the background to enhance the foreground
    shadow_removed = cv2.subtract(background, gray)

    # Normalize the image to the full 0-255 range
    shadow_removed = cv2.normalize(shadow_removed, None, 0, 255, cv2.NORM_MINMAX)

    # Convert to 3-channel image
    shadow_removed_color = cv2.cvtColor(shadow_removed, cv2.COLOR_GRAY2BGR)

    # Get image dimensions
    h, w = shadow_removed_color.shape[:2]

    # Define polygon to cover top 1/4 of the image height
    y_end = int(h * 0.28)
    pts = np.array([
        [0, 0],
        [w, 0],
        [w, y_end],
        [0, y_end]
    ], np.int32).reshape((-1, 1, 2))

    # Fill the polygon with black
    cv2.fillPoly(shadow_removed_color, [pts], color=(0, 0, 0))

    # Save the processed image
    filename = os.path.basename(image_path)
    output_path = os.path.join(output_folder, f"processed_{filename}")
    cv2.imwrite(output_path, shadow_removed_color)

print("Processing completed. Processed images saved to:", output_folder)


Processing completed. Processed images saved to: C:\New folder\Desktop\segregated_top_left_defect\processed
